# The no-free-lunch theorem for supervised machine learning in action!

<img src="files/schaffer.jpeg">

## The magnum opus in this area is "The Lack of a priori Distinctions Between Learning Algorithms" by David H. Wolpert (1995). This paper is simpler.

# QUESTION: Can we have a learning algorithm that we know will perform well on ANY supervised classification task, without knowing anything about the task?

## Central result: the "law of conservation of generalization performance."

<img src="files/theorem.jpeg">

### - S -> "learning situation" (D, C, n)
### - D -> sampling distribution for attribute vectors
### - n -> number of samples in training set
### - C -> function mapping attribute vectors to labels (or more generally distributions over labels).

In [1]:
import pandas as pd
import numpy as np
import re

## Specify the parameters. We are creating a binary classification problem with some number of binary features and a single binary target.

In [2]:
num_features = 3 # number of binary attributes
n = 5 # number of training vectors to sample
distribution = 'uniform'
if distribution == 'uniform':
    D = [1/(2**num_features) for x in range(2**num_features)]
elif distribution == 'random':
    D = np.random.rand(2**num_features)
    D = D/sum(D)

## Create the list of possible attribute vectors.

In [3]:
A = {mn: [int(x) for x in bin(mn)[2:].zfill(num_features)] for mn in range(2**num_features)}
A

{0: [0, 0, 0],
 1: [0, 0, 1],
 2: [0, 1, 0],
 3: [0, 1, 1],
 4: [1, 0, 0],
 5: [1, 0, 1],
 6: [1, 1, 0],
 7: [1, 1, 1]}

## Create the list of possible class vectors.

In [4]:
C = {mn: [int(x) for x in bin(mn)[2:].zfill(2**num_features)] for mn in range(2**len(A))}

In [5]:
C

{0: [0, 0, 0, 0, 0, 0, 0, 0],
 1: [0, 0, 0, 0, 0, 0, 0, 1],
 2: [0, 0, 0, 0, 0, 0, 1, 0],
 3: [0, 0, 0, 0, 0, 0, 1, 1],
 4: [0, 0, 0, 0, 0, 1, 0, 0],
 5: [0, 0, 0, 0, 0, 1, 0, 1],
 6: [0, 0, 0, 0, 0, 1, 1, 0],
 7: [0, 0, 0, 0, 0, 1, 1, 1],
 8: [0, 0, 0, 0, 1, 0, 0, 0],
 9: [0, 0, 0, 0, 1, 0, 0, 1],
 10: [0, 0, 0, 0, 1, 0, 1, 0],
 11: [0, 0, 0, 0, 1, 0, 1, 1],
 12: [0, 0, 0, 0, 1, 1, 0, 0],
 13: [0, 0, 0, 0, 1, 1, 0, 1],
 14: [0, 0, 0, 0, 1, 1, 1, 0],
 15: [0, 0, 0, 0, 1, 1, 1, 1],
 16: [0, 0, 0, 1, 0, 0, 0, 0],
 17: [0, 0, 0, 1, 0, 0, 0, 1],
 18: [0, 0, 0, 1, 0, 0, 1, 0],
 19: [0, 0, 0, 1, 0, 0, 1, 1],
 20: [0, 0, 0, 1, 0, 1, 0, 0],
 21: [0, 0, 0, 1, 0, 1, 0, 1],
 22: [0, 0, 0, 1, 0, 1, 1, 0],
 23: [0, 0, 0, 1, 0, 1, 1, 1],
 24: [0, 0, 0, 1, 1, 0, 0, 0],
 25: [0, 0, 0, 1, 1, 0, 0, 1],
 26: [0, 0, 0, 1, 1, 0, 1, 0],
 27: [0, 0, 0, 1, 1, 0, 1, 1],
 28: [0, 0, 0, 1, 1, 1, 0, 0],
 29: [0, 0, 0, 1, 1, 1, 0, 1],
 30: [0, 0, 0, 1, 1, 1, 1, 0],
 31: [0, 0, 0, 1, 1, 1, 1, 1],
 32: [0, 0, 1, 0, 

## Create a dataframe showing all of the possible attribute vectors, the class labels, and the sampling distribution.

In [6]:
data=pd.DataFrame(A).transpose().rename(columns = {cn: 'x'+str(cn) for cn in range(num_features)})
data['D']=D
data

,x0,x1,x2,D
0,0,0,0,0.125
1,0,0,1,0.125
2,0,1,0,0.125
3,0,1,1,0.125
4,1,0,0,0.125
5,1,0,1,0.125
6,1,1,0,0.125
7,1,1,1,0.125


## Randomly pick a training sample.

In [7]:
train = data.sample(n, weights=D).sort_index()
train

,x0,x1,x2,D
0,0,0,0,0.125
4,1,0,0,0.125
5,1,0,1,0.125
6,1,1,0,0.125
7,1,1,1,0.125


## Randomly select training labels.

In [8]:
train['labels']= np.random.randint(0, 2, len(train))
train

,x0,x1,x2,D,labels
0,0,0,0,0.125,0
4,1,0,0,0.125,1
5,1,0,1,0.125,0
6,1,1,0,0.125,1
7,1,1,1,0.125,0


## What are the possible 'truths' that are consistent with the training labels?

In [9]:
possible_truths = []
possible_truths_indices = []
for c in C:
    if pd.Series(C[c]).loc[train.index].tolist() == train['labels'].tolist():
        print(c, C[c])
        possible_truths.append(C[c])
        possible_truths_indices.append(c)

10 [0, 0, 0, 0, 1, 0, 1, 0]
26 [0, 0, 0, 1, 1, 0, 1, 0]
42 [0, 0, 1, 0, 1, 0, 1, 0]
58 [0, 0, 1, 1, 1, 0, 1, 0]
74 [0, 1, 0, 0, 1, 0, 1, 0]
90 [0, 1, 0, 1, 1, 0, 1, 0]
106 [0, 1, 1, 0, 1, 0, 1, 0]
122 [0, 1, 1, 1, 1, 0, 1, 0]


## If we don't know anything about the problem we're trying to solve, that's equivalent to each of these possible truths being equally likely! Let's compute the off-training set generalization performance for each of these possible truths, and each possible prediction.

In [10]:
off_train = data.drop(train.index)
off_train

,x0,x1,x2,D
1,0,0,1,0.125
2,0,1,0,0.125
3,0,1,1,0.125


In [11]:
x_cols = [c for c in off_train.columns if re.match('x', c)]

In [12]:
possible_truths

[[0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 1, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 0, 1, 0]]

In [13]:
for pred in possible_truths:
    off_train_pred = [pred[i] for i in off_train.index]
    weights = data['D'].loc[off_train.index].astype(float).tolist()
    print("prediction:", off_train_pred)
    averages = []
    for truth in possible_truths:
        off_train_truth = [truth[i] for i in off_train.index]
        correct = ([i == j for i, j in zip(off_train_truth, off_train_pred)])
        generalized_performance = [float(c) - 0.5 for c in correct]
        average = sum(generalized_performance)/len(generalized_performance)
        print("\ttruth:", off_train_truth, ", correct:", correct, ", GP:", generalized_performance, ", Avg. GP:", average)
        averages.append(average)
    print("\tOverall average:", sum(averages)/len(averages))

prediction: [0, 0, 0]
	truth: [0, 0, 0] , correct: [True, True, True] , GP: [0.5, 0.5, 0.5] , Avg. GP: 0.5
	truth: [0, 0, 1] , correct: [True, True, False] , GP: [0.5, 0.5, -0.5] , Avg. GP: 0.16666666666666666
	truth: [0, 1, 0] , correct: [True, False, True] , GP: [0.5, -0.5, 0.5] , Avg. GP: 0.16666666666666666
	truth: [0, 1, 1] , correct: [True, False, False] , GP: [0.5, -0.5, -0.5] , Avg. GP: -0.16666666666666666
	truth: [1, 0, 0] , correct: [False, True, True] , GP: [-0.5, 0.5, 0.5] , Avg. GP: 0.16666666666666666
	truth: [1, 0, 1] , correct: [False, True, False] , GP: [-0.5, 0.5, -0.5] , Avg. GP: -0.16666666666666666
	truth: [1, 1, 0] , correct: [False, False, True] , GP: [-0.5, -0.5, 0.5] , Avg. GP: -0.16666666666666666
	truth: [1, 1, 1] , correct: [False, False, False] , GP: [-0.5, -0.5, -0.5] , Avg. GP: -0.5
	Overall average: 0.0
prediction: [0, 0, 1]
	truth: [0, 0, 0] , correct: [True, True, False] , GP: [0.5, 0.5, -0.5] , Avg. GP: 0.16666666666666666
	truth: [0, 0, 1] , correct

## Implication

<img src="files/possible_impossible.jpeg">

## (Top row is possible, bottom rows are impossible.)

# "In short, empirical success in generalization is always due to problem selection. Although it is tempting to interpret such success as evidence in favour of a learner, it rather shows that the learner has been well applied."